<a href="https://colab.research.google.com/github/evansjeffm-99/NHL_LOGOS/blob/main/Django_accounts_models_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# accounts/models.py
from django.db import models
from django.contrib.auth.models import AbstractUser
from django.conf import settings # Recommended way to refer to AUTH_USER_MODEL
from django.db.models.signals import post_save
from django.dispatch import receiver

# Extend the built-in User model
class User(AbstractUser):
    """
    Custom User model extending Django's AbstractUser.
    Uses email as the unique identifier instead of username by default.
    """
    # Add any extra fields for the User model itself if needed
    # Example: make email required and unique for login
    email = models.EmailField(unique=True, blank=False, null=False)

    # Optional: Define email as the username field for authentication
    # USERNAME_FIELD = 'email'
    # REQUIRED_FIELDS = ['username'] # Fields required when creating user via createsuperuser

    def __str__(self):
        """String representation of the User model."""
        return self.username

# Profile model linked to the User model via a OneToOneField
class Profile(models.Model):
    """
    Stores additional user profile information, linked one-to-one with the User model.
    """
    # OneToOneField ensures each user has exactly one profile.
    # `related_name='profile'` allows accessing the profile from the user instance (e.g., user.profile)
    user = models.OneToOneField(
        settings.AUTH_USER_MODEL, # Reference the custom User model defined in settings
        on_delete=models.CASCADE,  # If the User is deleted, delete the Profile too
        related_name='profile'
    )
    # Optional avatar field, stores images in MEDIA_ROOT/avatars/
    avatar = models.ImageField(upload_to='avatars/', null=True, blank=True)
    # Optional text field for favorite team
    favorite_team = models.CharField(max_length=100, blank=True)
    # Optional text field for user biography
    bio = models.TextField(blank=True)

    def __str__(self):
        """String representation of the Profile model."""
        return f"{self.user.username}'s Profile"

# --- Signals ---

# Use signals to automatically create/update related models when the User model changes.

@receiver(post_save, sender=settings.AUTH_USER_MODEL)
def create_user_profile(sender, instance, created, **kwargs):
    """
    Signal receiver function to automatically create a Profile instance
    whenever a new User instance is created.
    """
    # Check if a new User instance was actually created
    if created:
        # Create a new Profile linked to the new User
        Profile.objects.create(user=instance)
        # print(f"Profile created for user {instance.username}") # For debugging

@receiver(post_save, sender=settings.AUTH_USER_MODEL)
def save_user_profile(sender, instance, **kwargs):
    """
    Signal receiver function to save the associated Profile instance
    whenever the User instance is saved.
    This ensures that if the profile relationship somehow didn't exist,
    it gets created, or if other logic updates the profile via the user, it's saved.
    """
    try:
        # Access the related profile and save it.
        instance.profile.save()
        # print(f"Profile saved for user {instance.username}") # For debugging
    except Profile.DoesNotExist:
        # This case should ideally be handled by the create_user_profile signal,
        # but this provides a fallback just in case.
        # print(f"Profile did not exist for user {instance.username}, creating one.") # For debugging
        Profile.objects.create(user=instance)